# Modulos e Informe

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn import linear_model

Uso pandas y numpy para modificar los data set y dejarlos de una manera que pueda procesarlos para hacer los graficos, luego utilice plotly como libreria para representar gradficamente los datos, al final traje sklearn como libreria para el entrenamiento del modelo de ML.

# Creacion Data Frames

In [4]:
#Importo y creo todos los Data sets
df_ALUCarrera = pd.read_csv('Data/Alu_Carrera.csv', sep=';')
df_AluMateria = pd.read_csv('Data/Alu_Materia.csv', sep=';')
df_Carreras = pd.read_csv('Data/Carreras.csv', sep=';')
df_Finales = pd.read_csv('Data/Finales.csv', sep=';')
df_Materias = pd.read_csv('Data/Materias.csv', sep=';')

# LIMPIEZA

### Transformo el tipo de dato tiempo en los DF

In [3]:
#Arreglo el problema de que los datos de tipo tiempo estan como si fueran tipo: objeto en vez de tipo: DataTime
df_ALUCarrera[['Fecha_Ingreso','Fecha_Nac']] = df_ALUCarrera[['Fecha_Ingreso','Fecha_Nac']].apply(pd.to_datetime)
df_AluMateria[['Fecha_Inscrip','Fecha_1p','Fecha_2p','Fecha_TP','Fecha_Recup']] = df_AluMateria[['Fecha_Inscrip','Fecha_1p','Fecha_2p','Fecha_TP','Fecha_Recup']].apply(pd.to_datetime)
df_Finales[['Fecha_Examen','Fecha_Inscrip_Examen']] = df_Finales[['Fecha_Examen','Fecha_Inscrip_Examen']].apply(pd.to_datetime)

### Eliminacion de Carreras

In [4]:
# veo cuales carreras no tienen datos suficientes para merecer su analisis.
df_ALUCarrera['Id_Carrera'].value_counts()

5     963
6     687
7     676
2     641
4     616
1     582
3     506
9     494
13    334
10    270
8     204
17    127
14     57
60     37
16     14
39      8
18      3
11      2
12      2
40      2
41      1
59      1
Name: Id_Carrera, dtype: int64

In [5]:
listaDeCarreras = [11,60,18,39,59,40,41,16,12]
df_Carreras[df_Carreras['Id_Carrera'].isin(listaDeCarreras)]

,Id_Carrera,Nombre_Largo,Nombre_Corto,Abreviatura,Cuatrimestres
2,12,ANALISTA EN GESTION DE VENTAS,VENTAS,VTA,4
6,18,TECNICO SUPERIOR EN LOGISTICA,LOGÍSTICA,LOG,6
8,41,AGENTE DE TRANSPORTE ADUANERO Y DESPACHANTE DE...,AGENTE DE TRANSP. ADUANERO Y DESP. DE ADUANA,ATA,4
12,59,TÉCNICO SUPERIOR EN SERVICIOS DE PAISAJISMO,SERVICIOS DE PAISAJISMO,SERPAI,2
13,39,ADMINISTRADOR DE REDES,ADMINISTRADOR DE REDES,AR,4
14,16,TECNICO SUPERIOR EN GESTION JURIDICA,GESTIÓN JURíDICA,GJ,6
17,60,TÉC. SUP. EN DES. DE SIMULACIONES VIRTUALES Y ...,VIDEOJUEGOS,SVIRT,6
18,40,TÉCNICO EN SERVICIOS AGROPECUARIOS,TÉCNICO EN SERVICIOS AGROPECUARIOS,TSAGR,4
19,11,ASISTENTE GERENCIAL TRILINGÜE,TRILINGÜE,TRI,4


In [6]:
#Averiguo los alumnos que estan en cada carrera y en total son solo 70 datos los que voy a eliminar, lo cual me parece irrelevante.
df_ALUCarrera.loc[df_ALUCarrera['Id_Carrera'].isin(listaDeCarreras)].count()
#al mismo tiempo creo que tener 9 carreras mas, que no me aportarian valor a mi analisis si es molesto.

Id_Alumno           70
Id_Carrera          70
Semestre_Ingreso    70
Año_Ingreso         70
Id_Ctro_Ext         70
Id_Modalidad        70
Fecha_Ingreso       70
Activo              70
Sexo                70
Fecha_Nac           70
dtype: int64

In [7]:
#Elimino todo registro de las carreras que tienen datos ignorables
df_Carreras = df_Carreras[~df_Carreras['Id_Carrera'].isin(listaDeCarreras)]
df_Finales = df_Finales[~df_Finales['Id_Carrera'].isin(listaDeCarreras)]
df_ALUCarrera = df_ALUCarrera[~df_ALUCarrera['Id_Carrera'].isin(listaDeCarreras)]
df_AluMateria = df_AluMateria[~df_AluMateria['Id_Carrera'].isin(listaDeCarreras)]

### Elimino Años que no me interesan de el DF AluMateria

In [8]:
df_AluMateria['Año'].describe()

count    130673.000000
mean       2007.038654
std           3.238610
min        1982.000000
25%        2005.000000
50%        2007.000000
75%        2009.000000
max        2021.000000
Name: Año, dtype: float64

In [9]:
# Elimino los datos que no esten entre 2001 y 2010 ya que si los graficamos hay un monton de datos faltantes en varias carreras. 
df_AluMateria.loc[(df_AluMateria['Año']<2001) | (df_AluMateria['Año']>2010),['Fecha_Inscrip','Año']]=np.nan

In [10]:
#Verifico
df_AluMateria['Año'].unique()

array([2006., 2005., 2007., 2001., 2002., 2004., 2003., 2009., 2010.,
         nan, 2008.])

### Columna Edad 

In [11]:
#Busco Vaciar todos los datos de la fecha de nacimiento que son un error ya que nadie menor a 17 podria estar cursando la carrera 
df_ALUCarrera.loc[(pd.DatetimeIndex(df_ALUCarrera['Fecha_Ingreso']).year - pd.DatetimeIndex(df_ALUCarrera['Fecha_Nac']).year)<17, 'Fecha_Nac'] = np.nan

In [12]:
#Creo una columna llamada edad 
df_ALUCarrera['Edad'] = pd.DatetimeIndex(df_ALUCarrera['Fecha_Ingreso']).year - pd.DatetimeIndex(df_ALUCarrera['Fecha_Nac']).year 

In [13]:
#Verifico que los cambios se hayan hecho bien
df_ALUCarrera['Edad'].unique()

array([19., 52., 48., 43., 20., 21., 38., 39., 37., 34., 35., 36., 23.,
       22., 26., 25., 18., 33., 41., 32., 31., 40., 30., 29., 28., 27.,
       24., nan, 17., 53.])

### Sucursal Cordoba

In [14]:
#Cuento los datos que tengo de cada sucursal
df_ALUCarrera['Id_Ctro_Ext'].value_counts()

CO    6146
SF       4
VA       2
VM       2
JM       2
AR       1
Name: Id_Ctro_Ext, dtype: int64

In [15]:
#Veo cuantos datos hay de cordoba
df_AluMateria['Ctro_Ext'].value_counts()

CO    129165
JM       851
VM       206
VA       139
SF       100
CP        90
RC        69
AR        32
HE        13
VD         8
Name: Ctro_Ext, dtype: int64

In [16]:
#Elimino todo lo que no sea de cordoba
df_AluMateria = df_AluMateria[df_AluMateria['Ctro_Ext']=='CO']
df_ALUCarrera = df_ALUCarrera[df_ALUCarrera['Id_Ctro_Ext']=='CO']

# Manipulacion y modificacion de datos

### Estado de alumno en la materia

Estado del alumno en la materia: vasicamente quiero revisar en que condicion se encuentra el alumno en cada fila de la materia de la cual tengo un dato sobre el quiero saber si es regular, promociono la materia o esta libre en la misma.

In [17]:
#Creo la columna y la lleno con un string arbitrario para poder modificarla luego
df_AluMateria['Condicion'] = 'Estado'

In [18]:
#Creo la condicion de promocionado.
df_AluMateria.loc[  (df_AluMateria['Nota_1p'] >= 7 ) & (df_AluMateria['Nota_1p'] != 11) 
                    & (df_AluMateria['Nota_2p'] >= 7 ) & (df_AluMateria['Nota_2p'] != 11)
                    & (df_AluMateria['Nota_TP'] >= 7 ) & (df_AluMateria['Nota_TP'] != 11)
                    , 'Condicion'] = 'Promocionado'
#Creo la condicion de Regular basandome en la de Promocionado y agregando algunas condiciones mas.
df_AluMateria.loc[  (df_AluMateria['Condicion'] !='Promocionado')
                    & ((df_AluMateria['Nota_TP'] >= 4) & (df_AluMateria['Nota_2p'] >= 4) & (df_AluMateria['Nota_1p'] >= 4) 
                        & (df_AluMateria['Nota_TP'] !=11) & (df_AluMateria['Nota_2p'] !=11) & (df_AluMateria['Nota_1p'] !=11)) 
                    | ((df_AluMateria['Nota_Recup']!= 11 ) & (df_AluMateria['Nota_Recup']>= 4 ))
                    , 'Condicion'] = "Regular"
#Creo la condicion de libre por descarte de la de Regular y Promocionado.
df_AluMateria.loc[  (df_AluMateria['Condicion'] != "Promocionado") & (df_AluMateria['Condicion'] !='Regular')
                    , 'Condicion'] = 'Libre'

### Recursante

Busco encontrar una relacion entre la taza de desercion de los alumnos y la cantidad de recursantes
spoiler: no encontre nada

In [19]:
#Uso shift para usar el dato de la fila anterior y lo comparo con el de la fila actual.
#Si son iguales es porque el alumno esta recursando esa materia.
df_AluMateria['Tipo']=np.where(   ((df_AluMateria['Id_Materia'].shift() == df_AluMateria['Id_Materia']))
                                & ((df_AluMateria['Id_Carrera'].shift() == df_AluMateria['Id_Carrera']))
                                & ((df_AluMateria['Id_Alumno'].shift() == df_AluMateria['Id_Alumno']))
                                    ,'Recursante', np.nan)

### Egresado, Cursante, Desertor
intento crear una nueva columna en la que se determine si un alumno ya se egreso de la carrera, si la abandono o si continua cursando la misma

In [20]:
df_AluMateria['Estado'] = 'Estado'

In [21]:
df_AluMateria = pd.merge(df_AluMateria,
                        df_Carreras[['Id_Carrera','Cuatrimestres']],
                        how='inner',
                        on=['Id_Carrera'])

In [22]:
#Creo la condicion de Egresado
df_AluMateria.loc[ (df_AluMateria['Cuatrimestre'] == df_AluMateria['Cuatrimestres']), 'Estado'] = 'Egresado' 
#Creo la condicion de Cursando
df_AluMateria.loc[ (df_AluMateria['Estado'] != 'Egresado') &
                    (df_AluMateria['Cuatrimestre'] != df_AluMateria['Cuatrimestres']) &
                    (df_AluMateria['Año'] == 2010) &
                    (df_AluMateria['Semestre']==2), 'Estado'] = 'Cursando'
#Creo la condicion de Desertor
df_AluMateria.loc[(df_AluMateria['Estado']!='Egresado') &
                    (df_AluMateria['Estado']!='Cursando') &
                    (df_AluMateria['Cuatrimestre']!=df_AluMateria['Cuatrimestres'])&
                    (((df_AluMateria['Año']==2010)&(df_AluMateria['Semestre']==1))|
                    ((df_AluMateria['Año']!=2010))),'Estado']='Desertor'

# Graficas

### Cantidad de Alumnos de cada carrera x Año y semestre

In [23]:
df_Cantidad_Alumnos_Año = df_ALUCarrera.groupby(['Id_Carrera','Año_Ingreso','Semestre_Ingreso'])['Id_Alumno'].count().reset_index(name='Cantidad')
df_Cantidad_Alumnos_Año = pd.merge(df_Cantidad_Alumnos_Año,
                        df_Carreras[['Id_Carrera','Nombre_Corto']],
                        how='inner',
                        on=['Id_Carrera'])
px.line(df_Cantidad_Alumnos_Año, x="Año_Ingreso",y='Cantidad', color="Nombre_Corto", title='Cantidad de Alumnos de cada carrera x Año y semestre', facet_col='Semestre_Ingreso')

### Total de Alumnos X Año de la institucion separado en mujeres y varones

In [24]:
px.histogram(df_ALUCarrera, 
            x="Edad", 
            title='Edad de alumnos',
            color='Sexo',
            animation_frame=df_ALUCarrera['Año_Ingreso'].sort_values())

#### la tendencia entre 2001 y 2010 es que hubo una disminucion de la media de edad a medida que avanzan los años

### Total de Alumnos x Carrera separado entre mujeres y varones

In [25]:
df_Sexo_Alumno_xCarrera = df_ALUCarrera.groupby(['Id_Carrera','Sexo'])['Id_Alumno'].count().reset_index(name='Cantidad')
df_Sexo_Alumno_xCarrera = pd.merge(df_Sexo_Alumno_xCarrera,
                        df_Carreras[['Id_Carrera','Nombre_Corto']],
                        how='inner',
                        on=['Id_Carrera'])
px.bar(df_Sexo_Alumno_xCarrera, x='Nombre_Corto',y='Cantidad', color='Sexo')

#### Aqui buscaba determinar si hay alguna preferencia entre sexos por alguna carrera como en los casos de Analista de sistema y Agronegocios que la tendencia es que mas hombres eligen estas carreras en contraposicion a la carrera de Recursos Humanos y Relaciones Publicas.

### Total Alumnos x Carrera separado entre Egresado, Desertor y Cursando

In [34]:
df_Estado_Alumno_xCarrera = df_AluMateria.groupby(['Id_Carrera','Año','Estado'])['Id_Alumno'].count().reset_index(name='Cantidad')
df_Estado_Alumno_xCarrera = pd.merge(df_Estado_Alumno_xCarrera,
                        df_Carreras[['Id_Carrera','Nombre_Corto']],
                        how='inner',
                        on=['Id_Carrera'])
px.bar(df_Estado_Alumno_xCarrera, x='Nombre_Corto',y='Cantidad', color='Estado', 
        title='Total Alumnos x Carrera separado entre Egresado y Desertor', 
        animation_frame='Año', 
        barmode='group')

#### La taza de desercion a lo largo de los años es demasiado alta en todas la carreras. dentro de todo algo normal para un estudio superior, tendria que comparar con otras instituciones para verificiar si esta taza es demasiado alta o no

### Recursantes de Cada Materia x Año 

In [44]:
df_AluMateria_R = df_AluMateria.drop(['Id_Alumno','Condicion','Cuatrimestres','Estado','Semestre','Id_Carrera','Fecha_Inscrip', 'Ctro_Ext', 'Id_Modalidad', 'Id_Comision', 'Nota_1p',
       'Fecha_1p', 'Nota_2p', 'Fecha_2p', 'Nota_TP', 'Fecha_TP', 'Nota_Recup',
       'Fecha_Recup', 'Cuatrimestre'],axis=1)

In [48]:
Graf_R = df_AluMateria_R.groupby(['Id_Materia','Año'])['Tipo'].count().reset_index(name='cant')
Graf_R = pd.merge(Graf_R,
                df_Materias[['Id_Materia','Nombre']],
                how='inner',
                on=['Id_Materia'])
px.bar(Graf_R, x='Año',y='cant', color='Nombre', 
        title='Total Alumnos x Materia')


#### Aca buscaba determinar si habia una materia en especifico que haga que los alumnos deserten, pero no supe como hacerlo.

# ML

### Creo los DF que voy a usar en el modelo

In [29]:
#Separo en semestre 1 y 2 el Data set
df_AluMateria_1D = df_AluMateria[(df_AluMateria['Semestre']==1) & (df_AluMateria['Estado']=='Desertor')]
df_AluMateria_2D = df_AluMateria[(df_AluMateria['Semestre']==2) & (df_AluMateria['Estado']=='Desertor')]
df_AluMateria_1E = df_AluMateria[(df_AluMateria['Semestre']==1) & (df_AluMateria['Estado']=='Egresado')]
df_AluMateria_2E = df_AluMateria[(df_AluMateria['Semestre']==2) & (df_AluMateria['Estado']=='Egresado')]
df_AluMateria_TE = df_AluMateria[(df_AluMateria['Estado']=='Egresado')]
df_AluMateria_TD = df_AluMateria[(df_AluMateria['Estado']=='Desertor')]

In [30]:
#Elimino las columnas que no me sirven de los Data Sets
df_AluMateria_2D = df_AluMateria_2D.drop(['Id_Materia','Fecha_Inscrip','Ctro_Ext','Id_Modalidad','Id_Comision','Nota_1p','Fecha_2p','Fecha_1p','Nota_2p','Fecha_Recup','Cuatrimestres','Nota_TP','Fecha_TP','Nota_Recup','Cuatrimestre','Tipo','Condicion','Estado','Semestre'],axis=1)
df_AluMateria_1D = df_AluMateria_1D.drop(['Id_Materia','Fecha_Inscrip','Ctro_Ext','Id_Modalidad','Id_Comision','Nota_1p','Fecha_2p','Fecha_1p','Nota_2p','Fecha_Recup','Cuatrimestres','Nota_TP','Fecha_TP','Nota_Recup','Cuatrimestre','Tipo','Condicion','Estado','Semestre'],axis=1)
df_AluMateria_1E = df_AluMateria_1E.drop(['Id_Materia','Fecha_Inscrip','Ctro_Ext','Id_Modalidad','Id_Comision','Nota_1p','Fecha_2p','Fecha_1p','Nota_2p','Fecha_Recup','Cuatrimestres','Nota_TP','Fecha_TP','Nota_Recup','Cuatrimestre','Tipo','Condicion','Estado','Semestre'],axis=1)
df_AluMateria_2E = df_AluMateria_2E.drop(['Id_Materia','Fecha_Inscrip','Ctro_Ext','Id_Modalidad','Id_Comision','Nota_1p','Fecha_2p','Fecha_1p','Nota_2p','Fecha_Recup','Cuatrimestres','Nota_TP','Fecha_TP','Nota_Recup','Cuatrimestre','Tipo','Condicion','Estado','Semestre'],axis=1)
df_AluMateria_TE = df_AluMateria_TE.drop(['Id_Materia','Fecha_Inscrip','Ctro_Ext','Id_Modalidad','Id_Comision','Nota_1p','Fecha_2p','Fecha_1p','Nota_2p','Fecha_Recup','Cuatrimestres','Nota_TP','Fecha_TP','Nota_Recup','Cuatrimestre','Tipo','Condicion','Estado','Semestre','Id_Carrera'],axis=1)
df_AluMateria_TD = df_AluMateria_TD.drop(['Id_Materia','Fecha_Inscrip','Ctro_Ext','Id_Modalidad','Id_Comision','Nota_1p','Fecha_2p','Fecha_1p','Nota_2p','Fecha_Recup','Cuatrimestres','Nota_TP','Fecha_TP','Nota_Recup','Cuatrimestre','Tipo','Condicion','Estado','Semestre','Id_Carrera'],axis=1)

In [31]:
#Finalmente Creo los Data set que me van a servir para entrenar el modelo los diferencie entre semestre y egresado o desertor
df_ML1D = df_AluMateria_1D.groupby(['Id_Carrera','Año'])['Id_Alumno'].count().reset_index(name='Cantidad')
df_ML2D = df_AluMateria_2D.groupby(['Id_Carrera','Año'])['Id_Alumno'].count().reset_index(name='Cantidad')
df_ML1E = df_AluMateria_1E.groupby(['Id_Carrera','Año'])['Id_Alumno'].count().reset_index(name='Cantidad')
df_ML2E = df_AluMateria_2E.groupby(['Id_Carrera','Año'])['Id_Alumno'].count().reset_index(name='Cantidad')
df_MLTE = df_AluMateria_TE.groupby(['Año'])['Id_Alumno'].count().reset_index(name='Cantidad')
df_MLTD = df_AluMateria_TD.groupby(['Año'])['Id_Alumno'].count().reset_index(name='Cantidad')

### Modelo

In [33]:
print('-------------------------------------------------------------------------------------------------------')
print('Este es un modelo de ML que te permite pronosticar cuantos Egresados y Desertores tendras en cierto año')
print('-------------------------------------------------------------------------------------------------------')

print("Ingrese '1' si desea saber sobre Carrera o '2' sobre la Institucion")
Tipo = int(input("Ingrese '1' si desea saber sobre los alumnos de una Carrera en especifico o '2' sobre todos los alumnos de Institución: "))

if (Tipo == 2):
    print('-------------------------------------------------------------------------------------------------------')
    print('Ingrese el año que desea pronosticar')
    Año = int(input('Ingrese el año que desea pronosticar: '))

    print('-------------------------------------------------------------------------------------------------------')
    print("Ingrese '1' si quiere saber sobre los Egresado o '2' sobre los 'Desertores'")
    Estado = int(input("Ingrese '1' si quiere saber sobre los Egresado o '2' sobre los 'Desertores': "))
    
    if (Estado == 1):

        #Genero las variables para entrenar el modelo
        x = df_MLTE.drop(['Cantidad'],axis=1)
        y = df_MLTE['Cantidad']

        #Creo el modelo
        LR = linear_model.LinearRegression()
        LR.fit(x,y)
        y_pred = LR.intercept_ + LR.coef_* Año

        print('La estimacion es de {} Egresados para el Año {}'.format(str(int(y_pred)),Año))

    elif (Estado == 2):

        #Genero las variables para entrenar el modelo
        x = df_MLTD.drop(['Cantidad'],axis=1)
        y = df_MLTD['Cantidad']

        #Creo el modelo
        LR = linear_model.LinearRegression()
        LR.fit(x,y)
        y_pred = LR.intercept_ + LR.coef_* Año

        print('La estimacion es de {} Desertores para el Año {}'.format(str(int(y_pred)),Año))

    else:
        print('El Estado que ingresaste no es correcto')
elif (Tipo == 1):

    print('Las Carreras que tenemos son: ') 
    print('-----------------------------------------------------------------------')
    print(df_Carreras[['Id_Carrera','Nombre_Corto']])

    print('-----------------------------------------------------------------------')
    print("Ingrese el 'Id_Carrera' de la Carrera que desea conocer")
    Carrera = int(input("Ingrese el 'Id_Carrera' de la Carrera que desea conocer: "))

    print('-----------------------------------------------------------------------')
    print('Ingrese el semestre que desea Conocer')
    Semestre = int(input('Ingrese el semestre que desea Conocer: '))

    if (Semestre == 1):

        print('-------------------------------------------------------------------------------------------------------')
        print('Ingrese el año que desea pronosticar')
        Año = int(input('Ingrese el año que desea pronosticar: '))

        print('-------------------------------------------------------------------------------------------------------')
        print("Ingrese '1' si quiere saber sobre los Egresado o '2' sobre los 'Desertores'")
        Estado = int(input("Ingrese '1' si quiere saber sobre los Egresado o '2' sobre los 'Desertores': "))

        if (Estado == 1):

            #Genero las variables para entrenar el modelo
            x = df_ML1E[df_ML1E['Id_Carrera']== Carrera].drop(['Cantidad','Id_Carrera'],axis=1)
            y = df_ML1E[df_ML1E['Id_Carrera']== Carrera]['Cantidad']

            #Creo el modelo
            LR = linear_model.LinearRegression()
            LR.fit(x,y)
            y_pred = LR.intercept_ + LR.coef_* Año
            print('-----------------------------------------------------------------------')
            x = df_Carreras[df_Carreras['Id_Carrera']==Carrera]['Nombre_Corto']

            print('La estimacion es de {} Egresados para el Año {} en la carrera {}'.format(str(int(y_pred)),Año,str(x)))
        elif (Estado == 2):

            #Genero las variables para entrenar el modelo
            x = df_ML1D[df_ML1D['Id_Carrera']== Carrera].drop(['Cantidad','Id_Carrera'],axis=1)
            y = df_ML1D[df_ML1D['Id_Carrera']== Carrera]['Cantidad']

            #Creo el modelo
            LR = linear_model.LinearRegression()
            LR.fit(x,y)
            y_pred = LR.intercept_ + LR.coef_* Año

            print('-----------------------------------------------------------------------')
            x = df_Carreras[df_Carreras['Id_Carrera']==Carrera]['Nombre_Corto']
            print('La estimacion es de {} Egresados para el Año {} en la carrera {}'.format(str(int(y_pred)),Año,str(x)))
        else: 
            print('El estado Ingresado es Incorrecto')

    elif (Semestre == 2):
        print('-------------------------------------------------------------------------------------------------------')
        print('Ingrese el año que desea pronosticar')
        Año = int(input('Ingrese el año que desea pronosticar: '))
        
        print('-------------------------------------------------------------------------------------------------------')
        print("Ingrese '1' si quiere saber sobre los Egresado o '2' sobre los 'Desertores'")
        Estado = int(input("Ingrese '1' si quiere saber sobre los Egresado o '2' sobre los 'Desertores': "))
        if (Estado == 1):
            #Genero las variables para entrenar el modelo
            x = df_ML2E[df_ML2E['Id_Carrera']== Carrera].drop(['Cantidad','Id_Carrera'],axis=1)
            y = df_ML2E[df_ML2E['Id_Carrera']== Carrera]['Cantidad']
            LR = linear_model.LinearRegression()
            #Creo el modelo
            LR.fit(x,y)
            y_pred = LR.intercept_ + LR.coef_* Año
            print('-----------------------------------------------------------------------')
            x = df_Carreras[df_Carreras['Id_Carrera']==Carrera]['Nombre_Corto']
            print('La estimacion es de {} Egresados para el Año {} en la carrera {}'.format(str(int(y_pred)),Año,str(x)))
        elif (Estado == 2):
            #Genero las variables para entrenar el modelo
            x = df_ML2D[df_ML2D['Id_Carrera']== Carrera].drop(['Cantidad','Id_Carrera'],axis=1)
            y = df_ML2D[df_ML2D['Id_Carrera']== Carrera]['Cantidad']
            LR = linear_model.LinearRegression()
            #Creo el modelo
            LR.fit(x,y)
            y_pred = LR.intercept_ + LR.coef_* Año
            print('-----------------------------------------------------------------------')
            x = df_Carreras[df_Carreras['Id_Carrera']==Carrera]['Nombre_Corto']
            print('La estimacion es de {} Egresados para el Año {} en la carrera {}'.format(str(int(y_pred)),Año,str(x)))
        else: 
            print('El estado Ingresado es Incorrecto')
    else:
        print('El semestre ingreso es incorrecto')
else:
    print('El numero ingresado no se reconoce como carrera o institucion')


-------------------------------------------------------------------------------------------------------
Este es un modelo de ML que te permite pronosticar cuantos Egresados y Desertores tendras en cierto año
-------------------------------------------------------------------------------------------------------
Ingrese '1' si desea saber sobre Carrera o '2' sobre la Institucion
Las Carreras que tenemos son: 
-----------------------------------------------------------------------
    Id_Carrera                   Nombre_Corto
0            9                   AGRONEGOCIOS
1            3                     PUBLICIDAD
3            6            RELACIONES PÚBLICAS
4            7               RECURSOS HUMANOS
5            1           ANALISTA DE SISTEMAS
7           10                        TURISMO
9            4     ADMINISTRACIÓN DE EMPRESAS
10          13                    MULTIMEDIOS
11           5                 DISEÑO GRÁFICO
15           2                      MARKETING
16         